In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pomegranate as pg

In [ ]:
def case(prev, name, p):
  return [*prev, name, p], [*prev, f"!{name}", 1-p]


robo = pg.DiscreteDistribution({'robo': 0.001, '!robo': 0.999})
terremoto = pg.DiscreteDistribution({'terremoto': 0.002, '!terremoto': 0.998})

alarma = pg.ConditionalProbabilityTable(
  [
    *case(['robo', 'terremoto'], 'alarma', 0.95),
    *case(['robo', '!terremoto'], 'alarma', 0.94),
    *case(['!robo', 'terremoto'], 'alarma', 0.29),
    *case(['!robo', '!terremoto'], 'alarma', 0.001),
  ],
  [robo, terremoto])

juan = pg.ConditionalProbabilityTable(
  [
    *case(['alarma'], 'juan', 0.90),
    *case(['!alarma'], 'juan', 0.05),
  ],
  [alarma])

maria = pg.ConditionalProbabilityTable(
  [
    *case(['alarma'], 'maria', 0.70),
    *case(['!alarma'], 'maria', 0.01),
  ],
  [alarma])

s1 = pg.Node(robo, name="robo")
s2 = pg.Node(terremoto, name="terremoto")
s3 = pg.Node(alarma, name="alarma")
s4 = pg.Node(juan, name="juan")
s5 = pg.Node(maria, name="maria")

model = pg.BayesianNetwork("Avisos")
model.add_states(s1, s2, s3, s4, s5)
model.add_edge(s1, s3)
model.add_edge(s2, s3)
model.add_edge(s3, s4)
model.add_edge(s3, s5)

model.bake()

# Ejercicio 1

* Que Juan llame a la policía si María ha llamado a la policía.
* Que haya un terremoto si María ha llamado a la policía.
* Que haya un terremoto y un robo simultáneamente.
* Que haya un terremoto y un robo simultáneamente si Juan ha llamado a la policía.

In [ ]:
def predict(m, query):
  bs = model.predict_proba(query)
  return {s.name: b.parameters[0][s.name] for s, b in zip(model.states, bs) if type(b) is not str}

In [ ]:
predict(model, {'maria':'maria'})

In [ ]:
beliefs = model.predict_proba({'juan': 'juan'})
b = {s.name: b.parameters[0][s.name] for s, b in zip(model.states, beliefs) if type(b) is not str}
b['robo'] + b['terremoto']

#p = {s.name: s.distribution.parameters[0][s.name] for s in model.states}
model.probability([
    ['robo', 'terremoto', None, 'juan', None],
    ['robo', 'terremoto', 'alarma', 'juan', None],
  ])


In [ ]:
print("probabilidad: ",robo.probability('robo') * terremoto.probability('terremoto'))
print("log prob: ",robo.log_probability('robo') + terremoto.log_probability('terremoto'))